# MORDM

In [1]:
# Import general python packages
import pandas as pd
import numpy as np
import seaborn as sns
import copy

# Import functions
from dike_model_function import DikeNetwork  # @UnresolvedImport
from problem_formulation import get_model_for_problem_formulation
from problem_formulation import sum_over, sum_over_time

# Loading in the necessary modules for EMA workbench and functions
from ema_workbench import (Model, MultiprocessingEvaluator, Scenario,
                           Constraint, ScalarOutcome)
from ema_workbench.util import ema_logging
from ema_workbench import save_results, load_results
from ema_workbench.em_framework.optimization import (EpsilonProgress)

In [2]:
# Loading in all the 17 objectives via predefined problem formulation 3
if __name__ == '__main__':
    ema_logging.log_to_stderr(ema_logging.INFO)
    dike_model, planning_steps = get_model_for_problem_formulation(6)

In [3]:
print(planning_steps)

[0 1 2 3 4]


In [4]:
# Replicate the objectives
for outcome in dike_model.outcomes:
    print(repr(outcome))
    

ArrayOutcome('A.1_Expected Annual Damage')
ArrayOutcome('A.1_Expected Number of Deaths')
ScalarOutcome('All Costs', variable_name=('A.1_Expected Annual Damage', 'A.2_Expected Annual Damage', 'A.3_Expected Annual Damage', 'A.4_Expected Annual Damage', 'A.5_Expected Annual Damage', 'A.1_Dike Investment Costs', 'A.2_Dike Investment Costs', 'A.3_Dike Investment Costs', 'A.4_Dike Investment Costs', 'A.5_Dike Investment Costs', 'RfR Total Costs', 'Expected Evacuation Costs'), function=<function sum_over at 0x000001F9DF8BE700>)
ArrayOutcome('A.2_Expected Annual Damage')
ArrayOutcome('A.2_Expected Number of Deaths')
ArrayOutcome('A.3_Expected Annual Damage')
ArrayOutcome('A.3_Expected Number of Deaths')
ArrayOutcome('A.4_Expected Annual Damage')
ArrayOutcome('A.4_Expected Number of Deaths')
ArrayOutcome('A.5_Expected Annual Damage')
ArrayOutcome('A.5_Expected Number of Deaths')


In [5]:
# Writing a function to create actor specific problem formulations
def problem_formulation_actor(problem_formulation_actor):
   
    # Load the model:
    function = DikeNetwork()
    # workbench model:
    model = Model('dikesnet', function=function)
    # Outcomes are all costs, thus they have to minimized:
    direction = ScalarOutcome.MINIMIZE
    
    model.uncertainties = uncertainties
    model.levers = levers
    
    cost_variables = []
    cost_variables.extend(
    [
        f"{dike}_{e}"
        for e in ["Expected Annual Damage", "Dike Investment Costs"]
        for dike in function.dikelist
    ])
    cost_variables.extend([f"RfR Total Costs"])
    cost_variables.extend([f"Expected Evacuation Costs"])


    if problem_formulation_actor == 4: #RWS
        model.outcomes.clear()
        model.outcomes = [
            ScalarOutcome('Expected Annual Damage',
                            variable_name=['{}_Expected Annual Damage'.format(dike)
                                                for dike in function.dikelist],
                            function=sum_over, kind=direction),

            ScalarOutcome('Total Investment Costs',
                            variable_name=['{}_Dike Investment Costs'.format(dike)
                                                for dike in function.dikelist] + ['RfR Total Costs'
                                                                                ] + ['Expected Evacuation Costs'],
                            function=sum_over, kind=direction),

            ScalarOutcome('Expected Number of Deaths',
                            variable_name=['{}_Expected Number of Deaths'.format(dike)
                                                for dike in function.dikelist],
                            function=sum_over, kind=direction)] 
    
    elif problem_formulation_actor == 5: # GELDERLAND
        model.outcomes.clear()
        model.outcomes = [
            ScalarOutcome('Expected Annual Damage A1', variable_name='A.1_Expected Annual Damage', function = sum_over, kind=direction),
            ScalarOutcome('Expected Annual Damage A2', variable_name='A.2_Expected Annual Damage', function = sum_over, kind=direction),
            ScalarOutcome('Expected Annual Damage A3', variable_name='A.3_Expected Annual Damage', function = sum_over, kind=direction),
            ScalarOutcome('Expected Number of Deaths in A1', variable_name='A.1_Expected Number of Deaths',function = sum_over, kind=direction),
            ScalarOutcome('Expected Number of Deaths in A2', variable_name='A.2_Expected Number of Deaths',function = sum_over, kind=direction),
            ScalarOutcome('Expected Number of Deaths in A3', variable_name='A.3_Expected Number of Deaths',function = sum_over, kind=direction),
            ScalarOutcome('Total Costs', variable_name=cost_variables, function = sum_over, kind=direction),
            ScalarOutcome('Aggregated Expected Number of Deaths A4-A5', variable_name = 
            ['A.4_Expected Number of Deaths', 'A.5_Expected Number of Deaths'], function = sum_over, kind=direction)]
    
    elif problem_formulation_actor == 6: # OVERIJSSEL
        model.outcomes.clear()
        model.outcomes = [
            ScalarOutcome('Expected Annual Damage A4', variable_name='A.4_Expected Annual Damage', function = sum_over, kind=direction),
            ScalarOutcome('Expected Annual Damage A5', variable_name='A.5_Expected Annual Damage', function = sum_over, kind=direction),            
            ScalarOutcome('Expected Number of Deaths in A4', variable_name='A.4_Expected Number of Deaths',function = sum_over, kind=direction),
            ScalarOutcome('Expected Number of Deaths in A5', variable_name='A.5_Expected Number of Deaths',function = sum_over, kind=direction),
            ScalarOutcome('Total Costs', variable_name=cost_variables, function = sum_over, kind=direction),
            ScalarOutcome('Aggregated Expected Number of Deaths A1-A3', variable_name =
            ['A.1_Expected Number of Deaths', 'A.2_Expected Number of Deaths', 
             'A.3_Expected Number of Deaths'], function = sum_over, kind=direction)]
    
    else:
        raise TypeError('unknown identifier')
    return model

In [6]:
# Replicate the uncertainties
uncertainties = dike_model.uncertainties
uncertainties = copy.deepcopy(dike_model.uncertainties)

In [7]:
levers = dike_model.levers 
levers = copy.deepcopy(dike_model.levers)

In [8]:
model = problem_formulation_actor(6)
for outcome in model.outcomes:
    print(repr((outcome)))

ScalarOutcome('Expected Annual Damage A4', variable_name=('A.4_Expected Annual Damage',), function=<function sum_over at 0x000001F9DF8BE700>)
ScalarOutcome('Expected Annual Damage A5', variable_name=('A.5_Expected Annual Damage',), function=<function sum_over at 0x000001F9DF8BE700>)
ScalarOutcome('Expected Number of Deaths in A4', variable_name=('A.4_Expected Number of Deaths',), function=<function sum_over at 0x000001F9DF8BE700>)
ScalarOutcome('Expected Number of Deaths in A5', variable_name=('A.5_Expected Number of Deaths',), function=<function sum_over at 0x000001F9DF8BE700>)
ScalarOutcome('Total Costs', variable_name=('A.1_Expected Annual Damage', 'A.2_Expected Annual Damage', 'A.3_Expected Annual Damage', 'A.4_Expected Annual Damage', 'A.5_Expected Annual Damage', 'A.1_Dike Investment Costs', 'A.2_Dike Investment Costs', 'A.3_Dike Investment Costs', 'A.4_Dike Investment Costs', 'A.5_Dike Investment Costs', 'RfR Total Costs', 'Expected Evacuation Costs'), function=<function sum_ove

In [9]:
print(len(model.outcomes))

6


In [10]:
for outcome in dike_model.outcomes:
    print(repr(outcome))

ArrayOutcome('A.1_Expected Annual Damage')
ArrayOutcome('A.1_Expected Number of Deaths')
ScalarOutcome('All Costs', variable_name=('A.1_Expected Annual Damage', 'A.2_Expected Annual Damage', 'A.3_Expected Annual Damage', 'A.4_Expected Annual Damage', 'A.5_Expected Annual Damage', 'A.1_Dike Investment Costs', 'A.2_Dike Investment Costs', 'A.3_Dike Investment Costs', 'A.4_Dike Investment Costs', 'A.5_Dike Investment Costs', 'RfR Total Costs', 'Expected Evacuation Costs'), function=<function sum_over at 0x000001F9DF8BE700>)
ArrayOutcome('A.2_Expected Annual Damage')
ArrayOutcome('A.2_Expected Number of Deaths')
ArrayOutcome('A.3_Expected Annual Damage')
ArrayOutcome('A.3_Expected Number of Deaths')
ArrayOutcome('A.4_Expected Annual Damage')
ArrayOutcome('A.4_Expected Number of Deaths')
ArrayOutcome('A.5_Expected Annual Damage')
ArrayOutcome('A.5_Expected Number of Deaths')


In [11]:
reference_values = {
    "Bmax": 175,
    "Brate": 1.5,
    "pfail": 0.5,
    "ID flood wave shape": 4,
    "planning steps": 2,
}
reference_values.update({f"discount rate {n}": 3.5 for n in planning_steps})
refcase_scen = {}

for key in dike_model.uncertainties:
    name_split = key.name.split('_')
    if len(name_split) == 1:

        refcase_scen.update({key.name: reference_values[key.name]})
    else:
        refcase_scen.update({key.name: reference_values[name_split[1]]})
            
ref_scenario = Scenario('reference', **refcase_scen)

In [136]:
convergence_metrics = {EpsilonProgress()}
constraint = [Constraint("Total Costs", outcome_names="Total Costs", function=lambda x: max(0, x - 700000000))]

results_df = pd.DataFrame()
with MultiprocessingEvaluator(model) as evaluator:
    for _ in range(2):
       results = evaluator.optimize(nfe=2, searchover='levers',
                                     convergence=convergence_metrics,
                                     epsilons=[1]*len(model.outcomes), reference=ref_scenario,
                                     constraints=constraint)
        
        
save_results(results, 'Week23_MORDM_Reference_1000_PD6.tar.gz')


[MainProcess/INFO] pool started with 3 workers
100it [01:31,  1.09it/s]                                                       
[MainProcess/INFO] optimization completed, found 1 solutions
[MainProcess/INFO] terminating pool


TypeError: cannot concatenate object of type '<class 'tuple'>'; only Series and DataFrame objs are valid

In [130]:
from ema_workbench.em_framework.optimization import epsilon_nondominated, to_problem

problem = to_problem(model, searchover="levers")
epsilons = [0.05] * len(model.outcomes)
merged_archives = epsilon_nondominated(results_df, epsilons, problem)

TypeError: cannot concatenate object of type '<class 'tuple'>'; only Series and DataFrame objs are valid

In [121]:
(y1,t1) , (y2,t2) = results

In [127]:
t2

,epsilon_progress,nfe
0,0,0
1,3,100


In [123]:
y = y1+y2
y

,0_RfR 0,0_RfR 1,0_RfR 2,0_RfR 3,0_RfR 4,1_RfR 0,1_RfR 1,1_RfR 2,1_RfR 3,1_RfR 4,...,A.5_DikeIncrease 1,A.5_DikeIncrease 2,A.5_DikeIncrease 3,A.5_DikeIncrease 4,Expected Annual Damage A4,Expected Annual Damage A5,Expected Number of Deaths in A4,Expected Number of Deaths in A5,Total Costs,Aggregated Expected Number of Deaths A1-A3
0,1,1,1,0,1,1,1,0,0,1,...,14,12,10,11,0.0,3.569558e+06,0.0,0.000581,4.259133e+09,0.0


In [112]:
from ema_workbench.em_framework.optimization import epsilon_nondominated, to_problem

problem = to_problem(model, searchover="levers")
epsilons = [0.05] * len(model.outcomes)
merged_archives = epsilon_nondominated(results, epsilons, problem)

TypeError: cannot concatenate object of type '<class 'tuple'>'; only Series and DataFrame objs are valid

# Gelderland


In [139]:
model = problem_formulation_actor(5)
for outcome in model.outcomes:
    print(repr((outcome)))

ScalarOutcome('Expected Annual Damage A1-4', variable_name=('A.1_Expected Annual Damage', 'A.2_Expected Annual Damage', 'A.3_Expected Annual Damage', 'A.4_Expected Annual Damage'), function=<function sum_over at 0x000001F1775E7BA0>)
ScalarOutcome('Investment Costs A1-4', variable_name=('A.1_Dike Investment Costs', 'A.2_Dike Investment Costs', 'A.3_Dike Investment Costs', 'A.4_Dike Investment Costs'), function=<function sum_over at 0x000001F1775E7BA0>)
ScalarOutcome('Expected Number of Deaths in A1-4', variable_name=('A.1_Expected Number of Deaths', 'A.2_Expected Number of Deaths', 'A.3_Expected Number of Deaths', 'A.4_Expected Number of Deaths'), function=<function sum_over at 0x000001F1775E7BA0>)


In [ ]:
convergence_metrics = {EpsilonProgress()}
constraint = [Constraint("Total Costs", outcome_names="Total Costs", function=lambda x: max(0, x - 700000000))]

results_df = pd.DataFrame()
with MultiprocessingEvaluator(model) as evaluator:
    for _ in range(2):
       results = evaluator.optimize(nfe=2, searchover='levers',
                                     convergence=convergence_metrics,
                                     epsilons=[1]*len(model.outcomes), reference=ref_scenario,
                                     constraints=constraint)
        
        
save_results(results, 'Week23_MORDM_Reference_1000_PD6.tar.gz')
